In [360]:
import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.pipeline import Pipeline


from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Load data and features names

In [174]:
# load data
path_data = '../Data/'

#list of all data names
city_data = ['jc_listings','jc_locale',
             'nyc_listings','nyc_locale',
             'rio_listings','rio_locale']
                   
dfs = {}

# loop to load all the data with respective name
for city in city_data:
    dfs[city] = pd.read_pickle(path_data+city+'.pkl')    
    
for key,val in dfs.items():
    exec(key + '=val')

In [175]:
# delete dfs dictionary to free memory
del dfs

In [264]:
features = ['neighbourhood_cleansed', 'latitude','longitude','room_type','bedrooms', 'beds',
            'accommodates', 'review_scores_rating', 'review_scores_cleanliness',
            'review_scores_location', 'review_scores_value']

# Preprocesses Data

Selecting the features

In [326]:
# Jersey City data

jc_features = jc_listings[features]
jc_target = jc_listings.price

jc_features.info() , jc_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2028 entries, 0 to 2565
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   neighbourhood_cleansed     2028 non-null   object 
 1   latitude                   2028 non-null   float64
 2   longitude                  2028 non-null   float64
 3   room_type                  2028 non-null   object 
 4   bedrooms                   1858 non-null   float64
 5   beds                       1973 non-null   float64
 6   accommodates               2028 non-null   int64  
 7   review_scores_rating       1708 non-null   float64
 8   review_scores_cleanliness  1706 non-null   float64
 9   review_scores_location     1706 non-null   float64
 10  review_scores_value        1706 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 190.1+ KB
<class 'pandas.core.series.Series'>
Int64Index: 2028 entries, 0 to 2565
Series name: price
Non-Null C

(None, None)

In [327]:
# Rio data

rio_features = rio_listings[features]
rio_target = rio_listings.price

rio_features.info(), rio_target.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23248 entries, 0 to 26365
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   neighbourhood_cleansed     23248 non-null  object 
 1   latitude                   23248 non-null  float64
 2   longitude                  23248 non-null  float64
 3   room_type                  23248 non-null  object 
 4   bedrooms                   22119 non-null  float64
 5   beds                       23068 non-null  float64
 6   accommodates               23248 non-null  int64  
 7   review_scores_rating       17147 non-null  float64
 8   review_scores_cleanliness  16973 non-null  float64
 9   review_scores_location     16973 non-null  float64
 10  review_scores_value        16973 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 2.1+ MB


(None,
 <bound method Series.info of 0        3500.0
 1        5000.0
 2         681.0
 3        8999.0
 4         500.0
           ...  
 26361     115.0
 26362     149.0
 26363      60.0
 26364     320.0
 26365     657.0
 Name: price, Length: 23248, dtype: float64>)

In [328]:
# New York City data

nyc_features = nyc_listings[features]
nyc_target = nyc_listings.price

nyc_features.info(), nyc_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22618 entries, 0 to 39879
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   neighbourhood_cleansed     22618 non-null  object 
 1   latitude                   22618 non-null  float64
 2   longitude                  22618 non-null  float64
 3   room_type                  22618 non-null  object 
 4   bedrooms                   20569 non-null  float64
 5   beds                       22156 non-null  float64
 6   accommodates               22618 non-null  int64  
 7   review_scores_rating       19108 non-null  float64
 8   review_scores_cleanliness  19083 non-null  float64
 9   review_scores_location     19083 non-null  float64
 10  review_scores_value        19083 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 2.1+ MB
<class 'pandas.core.series.Series'>
Int64Index: 22618 entries, 0 to 39879
Series name: price
Non-Null

(None, None)

Processing pipeline for numeric and categorical data

In [313]:
# Data Processing Pipeline

num_atributes = ['latitude','longitude', 'bedrooms', 'beds',
            'accommodates', 'review_scores_rating', 'review_scores_cleanliness',
            'review_scores_location', 'review_scores_value']

cat_atributes = ['neighbourhood_cleansed', 'room_type']

num_steps = [("imputer", SimpleImputer(missing_values=np.nan, strategy="median")), 
            ("scaler", StandardScaler())]

cat_steps = [("one_hot", OneHotEncoder(handle_unknown= 'ignore'))]

# data pipeline

num_pipeline = Pipeline(num_steps)
cat_pipeline = Pipeline(cat_steps)

preprocessor = ColumnTransformer([
                ("num", num_pipeline, num_atributes),
                ("cat", cat_pipeline, cat_atributes)])


# Jersey City

## Train test split and Base Models fit

In [443]:
# train test split
# JC

X = jc_features
y = np.log(jc_target)

jc_X_train,jc_X_test,jc_y_train,jc_y_test = train_test_split(X,y, test_size=0.3, random_state=27)

In [425]:
# Models fit

pipe_1 = Pipeline(steps = [("preprocess",preprocessor),("model", Lasso(alpha=.1))])
pipe_2 = Pipeline(steps = [("preprocess",preprocessor),("model", LinearRegression())])
pipe_3 = Pipeline(steps = [("preprocess",preprocessor),("model", RandomForestRegressor())])
pipe_4 = Pipeline(steps = [("preprocess",preprocessor),("model", xgb.XGBRegressor(eval_metric = 'rmse'))])

jc_Lasso = pipe_1.fit(jc_X_train, jc_y_train)
jc_LR = pipe_2.fit(jc_X_train, jc_y_train)
jc_RFR = pipe_3.fit(jc_X_train, jc_y_train)
jc_XGB = pipe_4.fit(jc_X_train, jc_y_train)



# Models Evaluation

In [426]:
# Model valuation

model_score = [jc_Lasso.score(jc_X_test, jc_y_test), 
               jc_LR.score(jc_X_test, jc_y_test), 
               jc_RFR.score(jc_X_test, jc_y_test),
               jc_XGB.score(jc_X_test, jc_y_test)]

predictions = (pipe_1.predict(jc_X_test), pipe_2.predict(jc_X_test), pipe_3.predict(jc_X_test),  pipe_4.predict(jc_X_test))

jc_rmse = []
for pred in predictions:
    jc_rmse.append(np.sqrt(mean_squared_error(np.exp(jc_y_test),np.exp(pred))))


In [427]:
print(model_score, '\n',jc_rmse)

[0.5477949585905698, 0.6351097381033965, 0.7505826186448199, 0.7369438567703941] 
 [302.2103659249311, 263.3429525934947, 224.30621776018404, 220.85283044485166]


Best models are Random Forest Regressor and XGB; We will optimize these two models

# Grid search and cross validation

In [428]:
# XGB GridSearchCV
xgb_params = {'model__n_estimators': range(50,250,25),
              'model__max_depth' : range(2,10,1),
              "model__min_child_weight":range(1, 5,1),
              'model__learning_rate': [0.1, 0.01]}

xgb_grid = GridSearchCV(estimator = pipe_4,
                        param_grid = xgb_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

xgb_grid.fit(jc_X_train, jc_y_train)

jc_xgb_grid = xgb_grid.best_estimator_

In [429]:
print(xgb_grid.best_params_, '\n', xgb_grid.best_score_)

{'model__learning_rate': 0.1, 'model__max_depth': 4, 'model__min_child_weight': 4, 'model__n_estimators': 150} 
 -0.15875810827216283


In [387]:
# Random Forest GridSearchCV

rfr_params = {'model__n_estimators': range(50,250,25),
              'model__max_features': ['sqrt', 'log2', None],
              'model__max_depth' : range(2,10,1),
              'model__min_samples_split' : range(2,10,1)}

rfr_grid = GridSearchCV(estimator = pipe_3,
                        param_grid = rfr_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

rfr_grid.fit(jc_X_train, jc_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['latitude',
                                                                          'longitude',
                                                                          'bedrooms',
                                                                          'beds',
                                                                     

In [388]:
print(rfr_grid.best_params_, '\n', rfr_grid.best_score_)

{'model__max_depth': 9, 'model__max_features': None, 'model__min_samples_split': 7, 'model__n_estimators': 75} 
 -0.16399165516925676


# Rio

## Train test split and Base Models fit

In [447]:
# train test split
# Rio

X = rio_features
y = np.log(rio_target)

rio_X_train,rio_X_test,rio_y_train,rio_y_test = train_test_split(X,y, test_size=0.3, random_state=27)

In [432]:
# Model fit

rio_Lasso = pipe_1.fit(rio_X_train, rio_y_train)
rio_LR = pipe_2.fit(rio_X_train, rio_y_train)
rio_RFR = pipe_3.fit(rio_X_train, rio_y_train)
rio_XGB = pipe_4.fit(rio_X_train, rio_y_train)

## Madels Evaluation

In [433]:
# Model valuation

model_score = [rio_Lasso.score(rio_X_test, rio_y_test), 
               rio_LR.score(rio_X_test, rio_y_test), 
               rio_RFR.score(rio_X_test, rio_y_test),
               rio_XGB.score(rio_X_test, rio_y_test)]

predictions = (pipe_1.predict(rio_X_test), pipe_2.predict(rio_X_test), pipe_3.predict(rio_X_test),  pipe_4.predict(rio_X_test))

rio_rmse = []
for pred in predictions:
    rio_rmse.append(np.sqrt(mean_squared_error(np.exp(rio_y_test),np.exp(pred))))

In [434]:
print(model_score, '\n',rio_rmse)

[0.31063110930823756, 0.3847420931339789, 0.5159466668490591, 0.5467374553953794] 
 [61670.36993046871, 456646.03195124416, 898.5844133169879, 895.731505013148]


Best models are Random Forest Regressor and XGB; We will optimize these two models

## Grid search and cross validation

In [435]:
# XGB GridSearchCV
xgb_params = {'model__n_estimators': range(50,250,25),
              'model__max_depth' : range(2,10,1),
              "model__min_child_weight":range(1, 5,1),
              'model__learning_rate': [0.1, 0.01]}

xgb_grid = GridSearchCV(estimator = pipe_4,
                        param_grid = xgb_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

xgb_grid.fit(rio_X_train, rio_y_train)

rio_xgb_grid = xgb_grid.best_estimator_

In [436]:
print(xgb_grid.best_params_, '\n', xgb_grid.best_score_)

{'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__min_child_weight': 4, 'model__n_estimators': 225} 
 -0.39523579162018735


In [392]:
# Random Forest GridSearchCV

rfr_params = {'model__n_estimators': range(50,250,25),
              'model__max_features': ['sqrt', 'log2', None],
              'model__max_depth' : range(2,10,1),
              'model__min_samples_split' : range(2,10,1)}

rfr_grid = GridSearchCV(estimator = pipe_3,
                        param_grid = rfr_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

rfr_grid.fit(rio_X_train, rio_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['latitude',
                                                                          'longitude',
                                                                          'bedrooms',
                                                                          'beds',
                                                                     

In [393]:
print(rfr_grid.best_params_, '\n', rfr_grid.best_score_)

{'model__max_depth': 9, 'model__max_features': None, 'model__min_samples_split': 8, 'model__n_estimators': 175} 
 -0.4203439067993762


# New York City

## Train test split and Base Models fit

In [448]:
# train test split
# NYC

X = nyc_features
y = np.log(nyc_target)

nyc_X_train,nyc_X_test,nyc_y_train,nyc_y_test = train_test_split(X,y, test_size=0.3, random_state=11)

In [395]:
# Model fit

nyc_Lasso = pipe_1.fit(nyc_X_train, nyc_y_train)
nyc_LR = pipe_2.fit(nyc_X_train, nyc_y_train)
nyc_RFR = pipe_3.fit(nyc_X_train, nyc_y_train)
nyc_XGB = pipe_4.fit(nyc_X_train, nyc_y_train)

## Models Evaluation

In [351]:
# Model valuation

model_score = [nyc_Lasso.score(nyc_X_test, nyc_y_test), 
               nyc_LR.score(nyc_X_test, nyc_y_test), 
               nyc_RFR.score(nyc_X_test, nyc_y_test),
               nyc_XGB.score(nyc_X_test, nyc_y_test)]

predictions = (pipe_1.predict(nyc_X_test), pipe_2.predict(nyc_X_test), pipe_3.predict(nyc_X_test),  pipe_4.predict(nyc_X_test))

nyc_rmse = []
for pred in predictions:
    nyc_rmse.append(np.sqrt(mean_squared_error(np.exp(nyc_y_test),np.exp(pred))))

In [352]:
print(model_score, '\n',nyc_rmse)

[0.36643840475311906, 0.6064484959655556, 0.687967085370149, 0.6743843920858883] 
 [369.67368645053665, 354.1918112749847, 335.5799557379308, 338.7298315145673]


## Grid search and cross validation

In [438]:
# Grid Search and cross validation:
# XGB GridSearchCV
xgb_params = {'model__n_estimators': range(50,250,25),
              'model__max_depth' : range(2,10,1),
              "model__min_child_weight":range(1, 5,1),
              'model__learning_rate': [0.1, 0.01]}

xgb_grid = GridSearchCV(estimator = pipe_4,
                        param_grid = xgb_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

xgb_grid.fit(nyc_X_train, nyc_y_train)

nyc_xgb_grid = xgb_grid.best_estimator_

In [439]:
print(xgb_grid.best_params_, '\n', xgb_grid.best_score_)

{'model__learning_rate': 0.1, 'model__max_depth': 9, 'model__min_child_weight': 2, 'model__n_estimators': 175} 
 -0.17606517179736997


In [399]:
# Random Forest GridSearchCV

rfr_params = {'model__n_estimators': range(50,250,25),
              'model__max_features': ['sqrt', 'log2', None],
              'model__max_depth' : range(2,10,1),
              'model__min_samples_split' : range(2,10,1)}

rfr_grid = GridSearchCV(estimator = pipe_3,
                        param_grid = rfr_params,
                        scoring = 'neg_mean_squared_error',
                        n_jobs = 10,
                        cv = 5)

rfr_grid.fit(nyc_X_train, nyc_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['latitude',
                                                                          'longitude',
                                                                          'bedrooms',
                                                                          'beds',
                                                                     

In [400]:
print(rfr_grid.best_params_, '\n', rfr_grid.best_score_)

{'model__max_depth': 9, 'model__max_features': None, 'model__min_samples_split': 7, 'model__n_estimators': 175} 
 -0.19203295852787533


XGBoosting Regression consistently out perfrom Random Forest Regression for the three case studies. Next we will build the best model for each city, measure its performance and save the model.

## Final Model Selection¶

## Model Performance

In [470]:
model_score = [round(jc_xgb_grid.score(jc_X_test, jc_y_test)*100,2), 
               round(rio_xgb_grid.score(rio_X_test, rio_y_test)*100,2), 
               round(nyc_xgb_grid.score(nyc_X_test, nyc_y_test)*100,2)]

predictions = (jc_xgb_grid.predict(jc_X_test), 
               rio_xgb_grid.predict(rio_X_test),
               nyc_xgb_grid.predict(nyc_X_test))
test_data = [jc_y_test, rio_y_test, nyc_y_test]
               
models_rmse = []
for pred, data in zip(predictions, test_data):
    models_rmse.append(np.sqrt(mean_squared_error(np.exp(data),np.exp(pred))))
    
data = {'R\u00b2': model_score, 'rmse': models_rmse}
cities = ['JC', 'Rio', 'NYC']

performance = pd.DataFrame(data,index = cities)

print(performance)

        R²        rmse
JC   75.37  220.627080
Rio  55.68  912.965461
NYC  69.16  337.548466


Grid Search hyperparameters optimization improved all XBoost models coeficient of determination. Jersey City root mean squared error (rmse) was marginaly improved while Rio and New York City were marginally worse. These results increase our confidense that the models aren't overfiting the current data. Note Rio rmse is larger than the rest because it is quoted in Brazilian Reais.

## Save best model object 

In [478]:
import datetime
# JC

jc_best_model = jc_xgb_grid
jc_best_model.version = '1.0'
jc_best_model.pandas_version = pd.__version__
jc_best_model.numpy_version = np.__version__
jc_best_model.xgb_version = xgb.__version__
jc_best_model.X_columns = [col for col in jc_X_train.columns]
jc_best_model.build_datetime = datetime.datetime.now()

# Rio

rio_best_model = rio_xgb_grid
rio_best_model.version = '1.0'
rio_best_model.pandas_version = pd.__version__
rio_best_model.numpy_version = np.__version__
rio_best_model.xgb_version = xgb.__version__
rio_best_model.X_columns = [col for col in jc_X_train.columns]
rio_best_model.build_datetime = datetime.datetime.now()

# NYC

nyc_best_model = jc_xgb_grid
nyc_best_model.version = '1.0'
nyc_best_model.pandas_version = pd.__version__
nyc_best_model.numpy_version = np.__version__
nyc_best_model.xgb_version = xgb.__version__
nyc_best_model.X_columns = [col for col in jc_X_train.columns]
nyc_best_model.build_datetime = datetime.datetime.now()

In [484]:
# save the model
import joblib
modelpath = '../models'

joblib.dump(jc_best_model, modelpath+'jc_pricing_model.pkl')
joblib.dump(rio_best_model, modelpath+'rio_pricing_model.pkl')
joblib.dump(nyc_best_model, modelpath+'nyc_pricing_model.pkl')

['../modelsnyc_pricing_model.pkl']